In [17]:
import pandas as pd       # para operar sobre dataframes
import matplotlib.pyplot as plt
import numpy as np        # para manipular vetores e realizar operações
import tensorflow as tf
from sklearn.preprocessing import StandardScaler # para escalonar os dados
from sklearn.metrics import mean_absolute_error, mean_squared_error # para obter métricas do modelo

from sklearn.model_selection import train_test_split


# Regressão simples com estimator na prática

Aplicando a regressão em um banco de dados obtido pela kaggle, chamada "House Sales in King County, USA". Pode ser obtida pelo link: https://www.kaggle.com/harlfoxem/housesalesprediction

Ou seja, vamos utilizar apenas o atributo metros_quadrados para predizer o preço da casa. 

O foco dessa atividade é trabalhar mais ativamente com as funções do numpy.

In [2]:
# Lendo o banco de dados
base = pd.read_csv("house_price.csv")

# Para confirmarmos se está tudo certo com o banco olhamos o começinho dele.
base.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [3]:
# Encontrando as dimensões do banco de dados
base.shape

(21613, 21)

Obtendo as variáveis de interesse...


In [8]:
# Obtendo os preços (que será o que queremos prever)
preco = base.iloc[:, 2:3].values # obtem a coluna de índice 2 do dataset e todas as suas linhas
print("Dimensões do preco", preco.shape)

# Obtendo os metros quadrados, que é o atributo que estamos usando para predição
metros_quad = base.iloc[:, 5:6].values # obtem a coluna de índice 2 do dataset e todas as suas linhas
print("Dimensões do metros_quad", metros_quad.shape)

Dimensões do preco (21613, 1)
Dimensões do metros_quad (21613, 1)


Agora vamos normalizar os dados de interesse para podermos passar para a regressão.


In [12]:
scaler_metros = StandardScaler()
scaler_precos = StandardScaler()

precos_escalados = scaler_precos.fit_transform(preco)
metros_escalados = scaler_metros.fit_transform(metros_quad)

# Visualizando os dados escalonados
print("Metros quadrados normalizados\n", metros_escalados)
print("\nPreços normalizados\n", precos_escalados)


Metros quadrados normalizados
 [[-0.97983502]
 [ 0.53363434]
 [-1.42625404]
 ...
 [-1.15404732]
 [-0.52252773]
 [-1.15404732]]

Preços normalizados
 [[-0.86671733]
 [-0.00568792]
 [-0.98084935]
 ...
 [-0.37586519]
 [-0.38158814]
 [-0.58588173]]


Agora vamos definir as colunas que vamos utilizar como atributos que estão no dataset. Nesse caso é apenas a coluna dos metros quadrados.

In [14]:
colunas_atributos = [tf.feature_column.numeric_column('metros_quadrados', shape=[1])]
colunas_atributos

[NumericColumn(key='metros_quadrados', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

Então, vamos aplicar o algoritmo LinearRegressor...



In [16]:
regressor = tf.estimator.LinearRegressor(feature_columns=colunas_atributos)
regressor

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\Natielle\\AppData\\Local\\Temp\\tmpfhr4__wq', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001FD68D01CC0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


Como já temos a aplicação do algortimo, agora vamos separar os dados em treinamento e teste.

In [19]:
# Aqui estamos separando 30% do dataset para testarmos e 70% para treinarmos o modelo
metros_train, metros_test, precos_train, precos_test = train_test_split(metros_escalados, precos_escalados, test_size = 0.3)

# Analisando as dimensões
print("metros_train: ", metros_train.shape)
print("metros_test: ", metros_test.shape)
print("precos_train: ", precos_train.shape)
print("precos_test: ", precos_test.shape)

metros_train:  (15129, 1)
metros_test:  (6484, 1)
precos_train:  (15129, 1)
precos_test:  (6484, 1)


Para realizarmos o treinamento, precisaremos fazer uma função do estimator.


In [20]:
# Função que vai retornar os dados de treino para o tensorflow
funcao_train = tf.estimator.inputs.numpy_input_fn({'metros_quadrados': metros_train}, precos_train,
                                                  batch_size=32,# pega em lotes de 32
                                                  num_epochs=None, # quantidade de vezes que vai rodar
                                                  shuffle=True) # pega ordem aleatória

# Função que vai retornar os dados de teste para o tensorflow
funcao_test = tf.estimator.inputs.numpy_input_fn({'metros_quadrados': metros_test}, precos_test,
                                                  batch_size=32,# pega em lotes de 32
                                                  num_epochs=1000, # quantidade de vezes que vai rodar
                                                  shuffle=False) # pega os registros em ordem 

In [21]:
# Vamos treinar de fato agora
regressor.train(input_fn=funcao_train, 
                steps=10000) 

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Use tf.cast instead.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\Natielle\AppData\Local\Temp\tmpfhr4__wq\model.ckpt.
INFO:tensorflow:loss = 33.366264, step = 1
INFO:tensorflow:global_step/sec: 615.136
INFO:tensorflow:loss = 9.818478, step = 101 (0.227 sec)
INFO:tensorflow:global_step/sec: 522.233
INFO:tensorflow:loss = 10.347807, step = 201 (0.106 sec)
INFO:tensorflow:global_step/sec: 1112.99
INFO:ten

INFO:tensorflow:global_step/sec: 1217.32
INFO:tensorflow:loss = 14.851602, step = 6301 (0.082 sec)
INFO:tensorflow:global_step/sec: 978.158
INFO:tensorflow:loss = 8.9751, step = 6401 (0.102 sec)
INFO:tensorflow:global_step/sec: 1214.49
INFO:tensorflow:loss = 25.067806, step = 6501 (0.095 sec)
INFO:tensorflow:global_step/sec: 1006.55
INFO:tensorflow:loss = 10.240131, step = 6601 (0.086 sec)
INFO:tensorflow:global_step/sec: 1077.5
INFO:tensorflow:loss = 34.39818, step = 6701 (0.095 sec)
INFO:tensorflow:global_step/sec: 1171.76
INFO:tensorflow:loss = 18.771063, step = 6801 (0.083 sec)
INFO:tensorflow:global_step/sec: 1139.41
INFO:tensorflow:loss = 12.756569, step = 6901 (0.090 sec)
INFO:tensorflow:global_step/sec: 1189.44
INFO:tensorflow:loss = 13.063949, step = 7001 (0.082 sec)
INFO:tensorflow:global_step/sec: 1016.85
INFO:tensorflow:loss = 8.620831, step = 7101 (0.098 sec)
INFO:tensorflow:global_step/sec: 955.925
INFO:tensorflow:loss = 13.505733, step = 7201 (0.108 sec)
INFO:tensorflow:

Notar que o steps roda a quantidade de vezes que ele tem que rodar dentro de um epochs.

Após treinarmos, podemos observar algumas métricas.


In [22]:
metricas_train = regressor.evaluate(input_fn=funcao_train, steps=10000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-05-06T00:58:44Z
INFO:tensorflow:Graph was finalized.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from C:\Users\Natielle\AppData\Local\Temp\tmpfhr4__wq\model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [1000/10000]
INFO:tensorflow:Evaluation [2000/10000]
INFO:tensorflow:Evaluation [3000/10000]
INFO:tensorflow:Evaluation [4000/10000]
INFO:tensorflow:Evaluation [5000/10000]
INFO:tensorflow:Evaluation [6000/10000]
INFO:tensorflow:Evaluation [7000/10000]
INFO:tensorflow:Evaluation [8000/10000]
INFO:tensorflow:Evaluation [9000/10000]
INFO:tensorflow:Evaluation [10000/10000]
INFO:tensorflow:Finished evaluation at 2020-05-06-00:58:53
INFO:tensorflow:Saving dict for global step 10000: average_loss = 0.4869371, global_step = 1

In [23]:
metricas_test = regressor.evaluate(input_fn=funcao_test, steps=10000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-05-06T00:59:29Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\Natielle\AppData\Local\Temp\tmpfhr4__wq\model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [1000/10000]
INFO:tensorflow:Evaluation [2000/10000]
INFO:tensorflow:Evaluation [3000/10000]
INFO:tensorflow:Evaluation [4000/10000]
INFO:tensorflow:Evaluation [5000/10000]
INFO:tensorflow:Evaluation [6000/10000]
INFO:tensorflow:Evaluation [7000/10000]
INFO:tensorflow:Evaluation [8000/10000]
INFO:tensorflow:Evaluation [9000/10000]
INFO:tensorflow:Evaluation [10000/10000]
INFO:tensorflow:Finished evaluation at 2020-05-06-00:59:41
INFO:tensorflow:Saving dict for global step 10000: average_loss = 0.55519867, global_step = 10000, label/mean = -0.0031537923, loss = 17.766357, prediction/mean = 0.0071647414
INF

In [24]:
print("Métricas treinamento: \n", metricas_train)
print("Métricas teste: \n", metricas_test)

Métricas treinamento: 
 {'average_loss': 0.4869371, 'label/mean': 0.0014513357, 'loss': 15.581987, 'prediction/mean': 0.0019821704, 'global_step': 10000}
Métricas teste: 
 {'average_loss': 0.55519867, 'label/mean': -0.0031537923, 'loss': 17.766357, 'prediction/mean': 0.0071647414, 'global_step': 10000}


É possível notar que aa métrica loss é menor no treino em grande parte das vezes.

**Dado que já fizemos as separações do dataset, agora vamos prever alguns exemplos.**

In [27]:
# Criando novos exemplos para prevermos
novas_casas = np.array([[800], [900], [1000]])
print("Novas casas: \n", novas_casas)

Novas casas: 
 [[ 800]
 [ 900]
 [1000]]


In [29]:
# Normalizando os dados para aplicarmos no algoritmo
novas_casas_escalonadas = scaler_metros.transform(novas_casas)
print("Novas casas normalizadas: \n", novas_casas_escalonadas)

Novas casas normalizadas: 
 [[-1.39358923]
 [-1.28470655]
 [-1.17582386]]


In [34]:
# Capturando os dados para ser passado para o tensorflow
funcao_previsao = tf.estimator.inputs.numpy_input_fn({'metros_quadrados': novas_casas_escalonadas},
                                                  shuffle=False) # pega os registros em ordem 
# realizando a previsão de fato
previsoes = regressor.predict(input_fn=funcao_previsao)

# observando o resultado
list(previsoes) # nota-se que nesse ponto os valores estão normalizados

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\Natielle\AppData\Local\Temp\tmpfhr4__wq\model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


[{'predictions': array([-0.98675025], dtype=float32)},
 {'predictions': array([-0.9093868], dtype=float32)},
 {'predictions': array([-0.83202326], dtype=float32)}]

In [38]:
# agora vamos visualizar os valores preditos sem a normalização
for p in regressor.predict(input_fn=funcao_previsao):
    print(scaler_precos.inverse_transform(p['predictions']))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\Natielle\AppData\Local\Temp\tmpfhr4__wq\model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
[177833.67]
[206235.23]
[234636.83]
